# `astroplan`: Interactive observation status

Design your own semi-real time display built on [astroplan](http://astroplan.readthedocs.org/) and [astropy](http://astropy.readthedocs.org/)!

In [2]:
target_name = 'KOI-1426'    # Resolved with Sesame
observatory_name = 'Apache Point Observatory' # Must be in astroplan

In [243]:
from astroplan import FixedTarget, Observer

from astropy.time import Time
import astropy.units as u
from astropy.coordinates import angle_utilities as util

from IPython.display import HTML, display

def table_to_html(table):
    """
    Make HTML table that prints with in a notebook 
    nicely via IPython.display.HTML, with length two rows
    """
    table_tags = "<table>\n{0}\n</table>"
    table_row_tags = "<tr><th>{0}</th><td>{1}</td></tr>"
    html = table_tags.format('\n'.join([table_row_tags.format(*row) 
                                        for row in table]))
    return HTML(html)

def hms_from_now(time):
    """
    Subtract ``time`` by the present time, return difference
    in hms format.
    """
    dt = time - Time.now()
    sign = '-' if dt < 0*u.min else '' # Determine sign
    
    return ('{0}{1:02.0f}:{2:02.0f}:{3:02.0f}'
            .format(sign, *map(abs, util.hours_to_hms(dt.to(u.hour).value))))

def time_table_to_html(table):
    """
    Make HTML table that prints with in a notebook 
    nicely via IPython.display.HTML, with length three rows
    """
    table_tags = "<table>\n{0}{1}\n</table>"
    table_header = "<tr><th>Time</th><th>UTC</th><th>UTC-now</th></tr>"
    table_row_tags = "<tr><th>{0}</th><td>{1}</td><td>{2}</td></tr>"
    html = table_tags.format(table_header,
                             '\n'.join([table_row_tags.format(row[0], 
                                                              row[1].utc.iso.split('.')[0], 
                                                              hms_from_now(row[1])) 
                                        for row in table]))
    return HTML(html)

target = FixedTarget.from_name(target_name)
obs = Observer.at_site(observatory_name)
time = Time.now()

# Basic target position table
hour_angle = obs.target_hour_angle(time, target)
local_sidereal_time = obs.local_sidereal_time(time)
altaz = obs.altaz(time, target)
parallactic_angle = obs.parallactic_angle(time, target)
moon_sep = obs.moon_altaz(time).separation(target.coord)
moon_illum = obs.moon_illumination(time)

sexagesimal_kwargs = dict(sep=':', pad=True)
position_table = [['UTC', time.utc.iso],
                  ['RA [hms]', target.ra.to_string(unit=u.hourangle, **sexagesimal_kwargs)], 
                  ['Dec [dms]', target.dec.to_string(unit=u.deg, **sexagesimal_kwargs)],
                  ['Altitude', '{0:.2f}$^\circ$'.format(altaz.alt.degree)],
                  ['Azimuth', '{0:.2f}$^\circ$'.format(altaz.az.degree)],
                  ['Zenith Ang.', '{0:.2f}$^\circ$'.format(altaz.zen.to(u.degree))],
                  ['Airmass [sec z]', '{0:.3f}'.format(altaz.secz)],
                  ['Parallactic Ang.', '{0:.2f}$^\circ$'.format(parallactic_angle.degree)],
                  ['Hour Ang. [hms]', hour_angle.wrap_at(180*u.deg).to_string(precision=0, **sexagesimal_kwargs)],
                  ['LST [hms]', local_sidereal_time.to_string(precision=0, **sexagesimal_kwargs)],
                  ['Moon separation [dms]', moon_sep.to_string(unit=u.deg, precision=0, **sexagesimal_kwargs)],
                  ['Moon illumination', '{0:.2f}'.format(moon_illum)]
                 ]

# Rise/set times table
next_sun_rise_civil = obs.midnight(time, which='next')
morning_civil = obs.twilight_morning_civil(time, which='next')
morning_astro = obs.twilight_morning_astronomical(time, which='next')
target_rise = obs.target_rise_time(time, target, which='next', horizon=30*u.deg)
target_set = obs.target_set_time(time, target, which='next', horizon=30*u.deg)
target_transit = obs.target_meridian_transit_time(time, target, which='next')

time_table = [['Target rise [alt>30$^\circ$]', target_rise],
              ['Target set [alt<30$^\circ$]', target_set],
              ['Target meridian transit', target_transit],
              ['Midnight', next_sun_rise_civil],
              ['Morning astro. twilight', morning_astro],
              ['Morning civil twilight', morning_civil]
             ]

# Display HTML tables in notebook
display(table_to_html(position_table))
display(time_table_to_html(time_table))

UTC,2015-09-15 05:17:22.911
RA [hms],18:52:50.198
Dec [dms],48:46:39.5198
Altitude,53.64$^\circ$
Azimuth,309.41$^\circ$
Zenith Ang.,36.36 deg$^\circ$
Airmass [sec z],1.242
Parallactic Ang.,-80.62$^\circ$
Hour Ang. [hms],02:56:42
LST [hms],21:49:32
Moon separation [dms],94:21:21


Time,UTC,UTC-now
Target rise [alt>30$^\circ$],2015-09-15 20:57:02,15:39:37
Target set [alt<30$^\circ$],2015-09-15 07:42:12,02:24:47
Target meridian transit,2015-09-16 02:17:39,21:00:14
Midnight,2015-09-15 06:59:08,01:41:44
Morning astro. twilight,2015-09-15 11:24:03,06:06:38
Morning civil twilight,2015-09-15 12:22:25,07:05:01
